Name: **Luong Nguyen**  
Student ID: **1504210**  

# Introduction to Deep Learning 

## Session07: RNN for recognizing sorted int sequences
____

## Import the packages needed for this assignment

In [1]:
# install keras on google colab
!pip install -q keras
# is GPU working?
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# import packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, 

Using TensorFlow backend.


## Generate data

We will randomly generate sequences of 100 integers, label 0 as unsorted and 1 as sorted. Also, the test set will have a different value interval from the train set so that we can check the generalization of the model. The idea is that if we feed these labeled sequences into a RNN, it can learn to predict if an integer sequence is sorted.

In [75]:
x_train = []
y_train = []
x_test = []
y_test = []

for i in range(1000):
  seq = np.random.randint(1000, size=100)
  sorted_seq = sorted(seq)
  x_train.append(seq)
  y_train.append(0)
  x_train.append(sorted_seq)
  y_train.append(1)
  
  seq = np.random.randint(10000, size=100)
  sorted_seq = sorted(seq)
  x_test.append(seq)
  y_test.append(0)
  x_test.append(sorted_seq)
  y_test.append(1)
  
  
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print("x_train shape: %s" % (str(x_train.shape)))
print("y_train shape: %s" % (str(y_train.shape)))
print("x_test shape: %s" % (str(x_test.shape)))
print("y_test shape: %s" % (str(y_test.shape)))

x_train shape: (2000, 1, 100)
y_train shape: (2000,)
x_test shape: (2000, 1, 100)
y_test shape: (2000,)


## Training

In [77]:
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=(1, 100)))
model.add(LSTM(32))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=16,
                    validation_split=0.2,  shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_29 (LSTM)               (None, 1, 32)             17024     
_________________________________________________________________
lstm_30 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 25,377
Trainable params: 25,377
Non-trainable params: 0
_________________________________________________________________
Train on 1600 samples, validate on 400 samples
Epoch 1/5
1600/1600 [==============================] - 4s 3ms/step - loss: 0.5247 - acc: 0.9406 - val_loss: 0.3906 - val_acc: 0.9975
Epoch 2/5
1600/1600 [==============================] - 1s 399us/step - loss: 0.2286 - acc: 0.9975 - val_loss: 0.0757 - val_acc: 0.9950
Epoch 3/5
1600/1600 [==============================] - 1s 402us/step - loss: 0.0390 - a

Evaluate the model on our test set

In [78]:
model.evaluate(x_test, y_test)

2000/2000 [==============================] - 0s 51us/step


[0.0026904334409628065, 0.999]

The model quickly learned and did perform well on the test set.

### Examples of predictions

In [79]:
np.random.shuffle(x_test)
x_test[:5]

array([[[ 884,  360, 2771, 6713, 2956, 1892, 1739, 6255, 9323, 3078,
         9438, 1013, 2627,  383, 2494, 6144, 9124, 5154, 9065, 4331,
         6809, 1516, 9140, 5177, 8950, 5344, 3154, 3753, 8299,  523,
         3186, 9178, 1396, 1002, 1880, 3395, 4227, 6152, 5518, 6122,
         5951, 2368, 8549, 2993, 6798, 8992, 2108, 5755, 7406, 5104,
          570, 9489, 6202, 7081, 5926, 9271, 8686, 1708, 9072, 1619,
         1672, 5435, 8449, 8113, 8229, 1250, 2349, 9008, 5029, 5969,
         6147, 8772, 9361,  644, 4721,  678, 2284, 9501, 2582, 5837,
         6377, 6775, 4808, 2455, 5009, 8631,  772, 1245,  425, 2003,
         2403,  142, 8620, 6966,  318, 8518, 3436, 7232, 7178, 9271]],

       [[1125, 4834, 5104, 9503, 7250, 4678, 6089, 8136, 5107, 9139,
         8085, 4289, 2777, 4875, 4076, 9303, 7954, 6635, 1644, 2548,
         2633, 8908, 1714, 4129, 1375, 2998, 1981, 7911, 8927, 3249,
         8378, 2157, 6015, 1335, 7265, 2730, 3460, 4229, 9920, 2917,
         1820, 2914, 2964, 7257

In [80]:
model.predict(x_test[:5])

array([[6.1775529e-04],
       [7.3131094e-05],
       [9.9970597e-01],
       [9.9970597e-01],
       [2.0419968e-05]], dtype=float32)

The model outputs a value close to 1 with a sorted sequence, and a value close to 0 with an unsorted sequence.